<a href="https://colab.research.google.com/github/pauloEzequiel/Sentiment_Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importando as bibliotecas necessárias a essa atividade
import os
import re
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [15]:
#Leitura dos arquivos da coleção ReLi
base_path_reli = 'ReLi-Lex'
train = []
files = [os.path.join(base_path_reli, f) for f in os.listdir(base_path_reli)]

for file in files:
    t = 'pos' if '_Positivos' in file else 'neg'

    with open(file, "r", encoding='iso8859-1') as content_file:
        content = content_file.read()
        all = re.findall('\[.*?\]',content)
        for w in all:
            train.append((w[1:-1], t))

In [16]:
#Transformação da coleção ReLi em dicionário, substituindo o rótulo neg para Negativo e pos para Positivo
#Dessa forma, os rótulos serão igualados a outra coleção
train_dict = {}
for x, y in train:
    if y == "neg":
        train_dict[x] = "Negativo"
    elif y == "pos":
        train_dict[x] = "Positivo"

In [17]:
#Para efeitos de estudo, é sempre bom saber com o que estamos lidando
print(type(train_dict), len(train_dict))

<class 'dict'> 597


In [21]:
#Transformando a coleção do ReLi em dataFrame
df_resenhas = pd.DataFrame(data=[[item[0], item[1]] for item in train_dict.items()], columns=['Text', 'Classificacao'])
df_resenhas.head(10)

,Text,Classificacao
0,admirável,Positivo
1,adorável,Positivo
2,agradável,Positivo
3,alegre,Positivo
4,alucinante,Positivo
5,apaixonante,Positivo
6,apoteótico,Positivo
7,ardente,Positivo
8,arrasador,Positivo
9,arrebatador,Positivo


In [22]:
#Leitura dos Tweets rotulados para análise de sentimentos
df_tweets = pd.read_csv('Tweets_Mg.csv',encoding='utf-8', usecols=["Text", "Classificacao"])
df_tweets.count()

Text             8199
Classificacao    8199
dtype: int64

In [23]:
#Verificando como estão os nossos dados
df_tweets.head()

,Text,Classificacao
0,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro
1,"� @ Governador Valadares, Minas Gerais https:/...",Neutro
2,"�� @ Governador Valadares, Minas Gerais https:...",Neutro
3,��� https://t.co/BnDsO34qK0,Neutro
4,��� PSOL vai questionar aumento de vereadores ...,Negativo


In [24]:
#Realizando a contagem de cada classe, em outros trabalhos talvez seja interessante ter classes balanceadas.
qtd_neutros   = df_tweets[df_tweets.Classificacao == "Neutro"]
qtd_positivos = df_tweets[df_tweets.Classificacao == 'Positivo']
qtd_negativos = df_tweets[df_tweets.Classificacao == 'Negativo']
print(10*"=","Tweets Neutros",10*"=")
print(qtd_neutros.count())
print()

print(10*"=","Tweets Positivos",10*"=")
print(qtd_positivos.count())
print()

print(10*"=","Tweets Negativos",10*"=")
print(qtd_negativos.count())

========== Tweets Neutros ==========
Text             2453
Classificacao    2453
dtype: int64

========== Tweets Positivos ==========
Text             3300
Classificacao    3300
dtype: int64

========== Tweets Negativos ==========
Text             2446
Classificacao    2446
dtype: int64


In [25]:
#Concatenando os dois DataSets
frames = [df_resenhas, df_tweets]
train_set = pd.concat(frames)

In [26]:
#Descrever a coleção ajuda a compreender possíveis melhorias
train_set.describe()


,Text,Classificacao
count,8796,8796
unique,6362,3
top,RT @AnaPaulaVolei: Mais 2 helicópteros!!A cara...,Positivo
freq,300,3677


In [27]:
#Como observado a duplicade de de registros, foi necessário a remoção de registros duplicados.
train_set.drop_duplicates(subset='Text', inplace=True)
train_set.describe()

,Text,Classificacao
count,6362,6362
unique,6362,3
top,admirável,Positivo
freq,1,3217


In [28]:
#Por desencargo de conciência, realizei a remoção de registros nulos, mas não resultou em nada
train_set.dropna(inplace=True)
train_set.describe()

,Text,Classificacao
count,6362,6362
unique,6362,3
top,admirável,Positivo
freq,1,3217


In [29]:
#Recontando os registros por classe, agora com a coleção concatenada e higienizada
qtd_neutros   = train_set[train_set.Classificacao == "Neutro"]
qtd_positivos = train_set[train_set.Classificacao == 'Positivo']
qtd_negativos = train_set[train_set.Classificacao == 'Negativo']
print(10*"=","Tweets Neutros",10*"=")
print(qtd_neutros.count())
print()

print(10*"=","Tweets Positivos",10*"=")
print(qtd_positivos.count())
print()

print(10*"=","Tweets Negativos",10*"=")
print(qtd_negativos.count())

========== Tweets Neutros ==========
Text             1974
Classificacao    1974
dtype: int64

========== Tweets Positivos ==========
Text             3217
Classificacao    3217
dtype: int64

========== Tweets Negativos ==========
Text             1171
Classificacao    1171
dtype: int64


In [30]:
# Remoção de Acentos e Caracteres Especiais - Padronização de Texto\n
dicionario_traducao = {'#': '', '{': '', '}': '', '>': '', '<': '', ',': '', ';': '',
                       '\"': '','@': '','&': '','®': '','¿': '','!': '','/': '','-':''}
train_set.replace(dicionario_traducao, regex=True, inplace=True)
train_set.describe()

,Text,Classificacao
count,6362,6362
unique,6361,3
top,E governo ainda quer indenizar a família dos b...,Positivo
freq,2,3217


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#Criação do modelo
from sklearn.pipeline import make_pipeline
modelo = make_pipeline(TfidfVectorizer(analyzer = "word"),
                       MultinomialNB())

classifier = GridSearchCV(estimator=modelo,
                          param_grid={
                              'multinomialnb__alpha':[0.1, 0.5, 1.0],
                              'multinomialnb__fit_prior':[True,False],
                              'tfidfvectorizer__ngram_range':[(1,1), (1,2), (2,2)]
                          },
                          n_jobs=4,
                          cv=10)

X = train_set['Text']
y = train_set['Classificacao']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

classifier.fit(X_train, y_train)

predicoes = classifier.predict(X_test)

np.mean(predicoes == y_test)

0.9023809523809524

In [32]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicoes))
print()
print("Melhores Parâmetros:", classifier.best_params_)

              precision    recall  f1-score   support

    Negativo       0.89      0.75      0.82       412
      Neutro       0.88      0.91      0.89       628
    Positivo       0.92      0.96      0.94      1060

    accuracy                           0.90      2100
   macro avg       0.90      0.87      0.88      2100
weighted avg       0.90      0.90      0.90      2100


Melhores Parâmetros: {'multinomialnb__alpha': 0.1, 'multinomialnb__fit_prior': True, 'tfidfvectorizer__ngram_range': (1, 2)}


In [33]:
#Persistindo o modelo para publicação futura em uma API
from joblib import dump, load
dump(classifier, 'SentimentAnalysisNB.pkl')

['SentimentAnalysisNB.pkl']

In [47]:
modelo = load('SentimentAnalysisNB.pkl')

validacao = ["Existem muitas formas de se aprender, basta querer.",
          "Há dias que o trabalho é uma verdadeira calamidade",
          "Hoje não estou muito bem",
          "Depois do culto você vai ver",
          "Vestidos de heróis dos pés a cabeça para levar um pouco de conhecimento para a humanidade",
          "A Net é muito boa, tem um excelente serviço de telefonia, sucesso de vendas",
          "excelente ação!"
          "O estado de Minas Gerais decretou calamidade financeira!!!"
         ]
resultado = modelo.predict(validacao)
predicted_proba = modelo.predict_proba(validacao)
for i in range(len(validacao)):
    print(str(i),"\t",resultado[i],"\t",predicted_proba[i])

0 	 Neutro 	 [0.4579421  0.50055913 0.04149877]
1 	 Neutro 	 [0.45148859 0.50890535 0.03960606]
2 	 Neutro 	 [0.1191235  0.62069443 0.26018207]
3 	 Neutro 	 [0.23336548 0.74324312 0.0233914 ]
4 	 Negativo 	 [0.63171978 0.06887401 0.29940621]
5 	 Positivo 	 [0.22403647 0.35633269 0.41963084]
6 	 Negativo 	 [0.84939369 0.13319822 0.01740809]
